In [3]:
import csv

fail_file = 'fail'
with open(fail_file, 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    fail_list = list(reader)
    fail_id = [x[0] for x in fail_list]
    fail_id = [int(x.replace('qid: ', '')) for x in fail_id]
    # print(fail_id)

import ast

alpha_file = '/dk/lcj_rag/pyserini-autofusion/experiments/runs/wq_test/run.dpr.wq-test.dpr.bm25.fusion.gt.all-alpha.nq--1.k-100.res.test'
class0 = 0
class1 = 0
with open(alpha_file, 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    lines = list(reader)
    # (qid, hits, softmax_hits, weights)
    for i in range(len(lines)):
        hits = lines[i][1]
        hits = hits.replace('hits: ', '')
        # hits = hits.to_list()
        hits = ast.literal_eval(hits)
        if i in fail_id and sum(hits):
            if hits[0] and hits[-1] == 0:
                class0 += 1
            if hits[0] == 0 and hits[-1]:
                class1 += 1
            print(i, hits)

print(f'class0: {class0}')
print(f'class1: {class1}')
    # lines = f.readlines()
    # for i, line in enumerate(lines):
    #     if i in fail_id:
    #         print(line)


120 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2]
210 [4, 3, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
244 [20, 17, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
283 [11, 11, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
299 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
309 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 3, 3, 4, 4]
324 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
394 [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
405 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
454 [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
462 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
476 [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
502 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
551 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
568 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [6]:
import csv
import pickle
import numpy as np

fail_file = 'fail'
with open(fail_file, 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    fail_list = list(reader)
    fail_id = [x[0] for x in fail_list]
    fail_id = [int(x.replace('qid: ', '')) for x in fail_id]
    # print(fail_id)

import ast

alpha_file = '/dk/lcj_rag/pyserini-autofusion/experiments/runs/wq_test/run.dpr.wq-test.dpr.bm25.fusion.gt.all-alpha.nq--1.k-100.res.test'

pkl_file = f'/dk/lcj_rag/pyserini-autofusion/experiments/runs/wq_train/wq_train_Xy-top{200}.pkl'
# # Load data using pickle
with open(pkl_file, 'rb') as f:
    Xy_np = pickle.load(f)
    X_np = Xy_np[:, :-1]
    X_np = np.hstack((X_np[:, 768:868], X_np[:, 968:1068]))
    print(X_np.shape)

class0 = []
class1 = []
class2 = []
class0_dist = []
class1_dist = []
class2_dist = []

def weighted_distance(np_x):
    w = np.array([1 / x for x in np.arange(1, len(np_x) + 1)])
    # w = w / np.sum(w)
    # print(w)
    return np.dot(np_x, w)

# weighted_distance(X_np[0, :10])

num_fail = len(fail_id)
with open(alpha_file, 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    lines = list(reader)
    # (qid, hits, softmax_hits, weights)
    for i in range(len(lines)):
        hits = lines[i][1]
        hits = hits.replace('hits: ', '')
        # hits = hits.to_list()
        hits = ast.literal_eval(hits)
        if i in fail_id and sum(hits):
            if hits[0] and hits[-1] == 0:
                class0.append((i, hits))
                class0_dist.append(weighted_distance(X_np[i, :100]))
            if hits[0] == 0 and hits[-1]:
                class1.append((i, hits))
                class1_dist.append(weighted_distance(X_np[i, :100]))
        else:
            class2.append((i, hits))
            class2_dist.append(weighted_distance(X_np[i, :100]))

print(f'avg: class0_dist: {np.mean(class0_dist)}')
print(f'minmax: class0_dist: {np.min(class0_dist), np.max(class0_dist)}')
print(f'std: class0_dist: {np.std(class0_dist)}')

print(f'avg: class1_dist: {np.mean(class1_dist)}')
print(f'minmax: class1_dist: {np.min(class1_dist), np.max(class1_dist)}')
print(f'std: class1_dist: {np.std(class1_dist)}')

print(f'avg: class2_dist: {np.mean(class2_dist)}')
print(f'minmax: class2_dist: {np.min(class2_dist), np.max(class2_dist)}')
print(f'std: class2_dist: {np.std(class2_dist)}')

print(f'class2_dist - class0_dist: {np.mean(class2_dist) - np.mean(class0_dist)}')
print(1 - (num_fail - 0) / 3610)

# last  delta_decay
# 5     0.01038106804426775
# 10    0.011653730710922572
# 25    0.012135627917595926
# 50    0.012989433003550044
# 100   0.013276806527493745
import csv
with open('class1.csv', 'w') as f:
    writer = csv.writer(f, delimiter='\t')
    # class1.append((i, hits))
    for id, _ in class1:
        writer.writerow([id])

(3260, 200)
avg: class0_dist: 417.05715864612705
minmax: class0_dist: (378.7735922503876, 456.94269286609466)
std: class0_dist: 21.237885183449713
avg: class1_dist: 417.80662655495104
minmax: class1_dist: (374.7037098004728, 452.3517646119559)
std: class1_dist: 23.285888752118936
avg: class2_dist: 418.98094525402837
minmax: class2_dist: (360.804031371419, 515.796079868365)
std: class2_dist: 21.53851457080751
class2_dist - class0_dist: 1.9237866079013202
0.9094182825484765
